In [2]:
import sagemaker
import boto3
import numpy as np                                
import pandas as pd                               
import os                                         
from sagemaker import get_execution_role
from sagemaker.processing import ProcessingInput, ProcessingOutput

# Get default bucket
session = sagemaker.Session()
default_bucket = session.default_bucket()
prefix = 'sagemaker/DEMO-xgboost-dm'

# Get SageMaker Execution Role
role = get_execution_role()
region = boto3.Session().region_name

In [3]:
# Define boto session and SageMaker Client
boto_session = boto3.Session(region_name=region)
sagemaker_client = boto_session.client(service_name='sagemaker', region_name=region)

In [4]:
%%writefile preprocessing.py
import argparse
import os

import numpy as np
import pandas as pd

# Helper method
def process(df):
    # Add two new indicators
    df["no_previous_contact"] = (df["pdays"] == 999).astype(int)
    df["not_working"] = df["job"].isin(["student", "retired", "unemployed"]).astype(int)
    columns = list(df.columns)
    
    toremove = ["emp.var.rate", "cons.price.idx", "cons.conf.idx", "euribor3m", "nr.employed"]
    columns = [x for x in columns if x not in toremove]
    
    # Keeping only columns that we need
    df = df[columns]
    
    # One hot encode
    df=pd.get_dummies(df)
    df = pd.concat([df['y_yes'], df.drop(['y_no', 'y_yes'], axis=1)], axis=1)
    df = df.sample(frac=1).reset_index(drop=True)
    return df

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--input-path", type=str, default="/opt/ml/processing")
    args, _ = parser.parse_known_args()
    
    base_dir = args.input_path

    df = pd.read_csv(
        f"{base_dir}/input/bank-additional-full.csv",
        header=0
    )
    
    # Call the helper method
    df = process(df)
    
    train, validation, test = np.split(df, [int(.7*len(df)), int(.85*len(df))])

    train.to_csv(f"{base_dir}/train/train.csv", header=False, index=False)
    validation.to_csv(f"{base_dir}/validation/validation.csv", header=False, index=False)
    test.to_csv(f"{base_dir}/test/test.csv", header=False, index=False)

Overwriting preprocessing.py


### Transformation

The transformations steps were made in DataWrangler & Features Store

In [5]:
local_data_path = "bank-additional-full.csv"

base_uri = f"s3://{default_bucket}/marketing"
input_data_uri = sagemaker.s3.S3Uploader.upload(
    local_path=local_data_path, 
    desired_s3_uri=base_uri,
)

In [6]:
from sagemaker.sklearn.processing import SKLearnProcessor


framework_version = "0.23-1"

sklearn_processor = SKLearnProcessor(
    framework_version=framework_version,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    base_job_name="sklearn-marketing-process",
    role=role,
    sagemaker_session=session
)

In [8]:
sklearn_processor.run(
    inputs=[
      ProcessingInput(source=input_data_uri, destination="/opt/ml/processing/input"),  
    ],
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/train"),
        ProcessingOutput(output_name="validation", source="/opt/ml/processing/validation"),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/test")
    ],
    code="preprocessing.py",
)


Job Name:  sklearn-marketing-process-2022-09-27-21-19-35-154
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-2-952750099313/marketing/bank-additional-full.csv', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-2-952750099313/sklearn-marketing-process-2022-09-27-21-19-35-154/input/code/preprocessing.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'train', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-us-east-2-952750099313/sklearn-marketing-process-2022-09-27-21-19-35-154/output/train', 'LocalPath': '/opt/ml/processing/train', 'S3UploadMode': 'EndOfJob'}}, {'OutputName